# Data Science and Machine Learning Capstone Project - Ammarsha

In this project I will analyze the 311 complaints related to Housing and Building in NYC.

First, let's ingest the dataset.

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_94a05f71faed4d5b84a2c32aeed24a76 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WixE9p6t4-6ZxAdl72POcvmGPCNXXRdz73w6L_XhkxCS',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_94a05f71faed4d5b84a2c32aeed24a76.get_object(Bucket='pythonammarsha-donotdelete-pr-rz2i5aqel02zrh',Key='311_Service_Requests_from_2010_to_Present_min.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

Question 1: The dataset contains complaints logged since what date?

We can sort dataframe by created date to find out the first data logged in:

In [ ]:
df.sort_values(by='Created Date')

Question 2: The dataset contains complaints logged till what date?

Using the head method:

In [ ]:
df.head()

Question 3: How many incidents have a missing Incident Address?

Using the isna() function to find the values of NaN in a column:

In [ ]:
df["Incident Address"].isna().sum()

For Question 4 and 5, let's retreive the Bronx and Queens data from PLUTO dataset.

In [ ]:
body = client_94a05f71faed4d5b84a2c32aeed24a76.get_object(Bucket='pythonammarsha-donotdelete-pr-rz2i5aqel02zrh',Key='BX_18v1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

bx = pd.read_csv(body)
bx.head()


In [ ]:
body = client_94a05f71faed4d5b84a2c32aeed24a76.get_object(Bucket='pythonammarsha-donotdelete-pr-rz2i5aqel02zrh',Key='QN_18v1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

qn = pd.read_csv(body)
qn.head()


Question 4: How many valid ZIP Codes exist in the Bronx PLUTO dataset?

Using the value_counts function we can quickly determine the number of ZIP codes in the Bronx

In [ ]:
bx['ZipCode'].value_counts().count()

Question 5: How many valid ZIP Codes exist in the Queens PLUTO dataset?

Using the same value_counts method from Question 4 we can determine unique ZIP Codes for Queens:

In [ ]:
qn['ZipCode'].value_counts().count()

### Section 2 - What is the Top Complaint Type

In this section we'd like to figure what kind of complaint the HPD should focus on.

Question 1: What is the total number of complaints that exist in the dataset?

Using the count function:

In [ ]:
df.count

Question 2: How many different Complaint Types can you find in the dataset, including duplicates entries of the same type?

Using the value counts we can determine how many unique complaints in the dataset

In [ ]:
df['Complaint Type'].value_counts().count()

Question 3: How many Elevator complaints can you find in the dataset?

Question 4: How many Electric complaints can you find in the dataset?

Question 5: Using 80,000 as a threshold, what complaint type(s) do you recommend the Department of Housing Preservation and Development of New York City address first? Select all that apply.

Again, use the value_counts function to count the complaints. This function will help us figuring out the answer for each question.

In [ ]:
df['Complaint Type'].value_counts()

Elevator complaints: 6,725 complaints

Electric complaints: 307,311 complaints

Complaints over 80,000: Heating and both of Heat/Hot Water

Let's make a graph out of it:

In [ ]:
import matplotlib.pyplot as plt
df['Complaint Type'].value_counts().plot(kind='bar', figsize=(10, 6))

plt.xlabel('Complaint Type')
plt.ylabel('Numbers of Occurence')
plt.title('Complaint occurence based on their type')

plt.show()

## Section 3 - What Areas Should the Agency Focus On?

We will use the dataset to determine whether HPD should focus on a certain borough, ZIP code, or street.

Question 1: For the complaint types that you selected in the previous module that had at least 80,000 complaints logged, which borough had the highest number of complaints submitted?

Question 2: For the complaint types that you selected in the previous module that had a total number that exceeded 80,000 complaints, which borough had the lowest number of complaints submitted?

Let's make a new dataframe where we group both "Heating" and "Heat/Hot Water" as both complaints are over 80,000.

In [ ]:
ht = df[['Complaint Type', 'Closed Date', 'Borough', 'Street Name', 'Incident Zip']]
ht.head()

In [ ]:
ht1 = ht.loc[df['Complaint Type'].isin(['HEAT/HOT WATER', 'HEATING'])]
ht1.head()

Let's find the Borough with most complaints using the value_counts function:

In [ ]:
ht1['Borough'].value_counts()

In [ ]:
ht1['Borough'].value_counts().plot(kind='barh', figsize=(10, 6))

plt.xlabel('Borough')
plt.ylabel('Numbers of Occurence')
plt.title('Heating and Hot Water Complaints in Five Borough of NYC')

plt.show()

Question 3: For the complaint types that you selected in the previous module that had at least 80,000 complaints logged, which ZIP code had the highest number of complainted submitted?

Let's again use the value_counts function to determine which ZIP code has the highest number of complaints.

In [ ]:
ht1['Incident Zip'].value_counts().head()

In [ ]:
ht1['Incident Zip'].value_counts().head().plot(kind='barh', figsize=(10, 6))

plt.xlabel('Zip Code')
plt.ylabel('Numbers of Occurence')
plt.title('Top 5 Area based on ZIP code withHeating and Hot Water Complaints in NYC')

plt.show()

Question 4: For the complaint types that you selected in the previous module that had at least 80,000 complaints logged, the address 89-21 Elmhurst Avenue had the highest number of complainted submitted?

Again, value_counts.

In [ ]:
ht1['Street Name'].value_counts().head()

Question 5: For the complaint types that you selected in the previous module that had at least 80,000 complaints logged, how many of the submitted tickets were closed?

Let's count the non null values from the Closed Date column.

In [ ]:
ht1['Closed Date'].count()

## Section 4 - What Is the Relationship between Housing Characteristics and Complaints?

In this section we will determine whether the problem identified (Heating and Hot Water Complaints) have an obvious relationship with any particular characteristic of the houses.

Question 1: Can you determine the age of the building from the PLUTO dataset?
    
Let's look at the BX dataset as The Bronx is the borough with most complaints.

In [ ]:
bx.head()

This is too much for us to handle, obviously. Let's reduce the column numbers so it's bearable to analyze.

In [ ]:
bx1 = bx[['Address', 'BldgArea', 'BldgDepth', 'BuiltFAR', 'CommFAR', 'FacilFAR', 'Lot', 'LotArea', 'LotDepth', 'NumBldgs', 'NumFloors', 'OfficeArea', 'ResArea', 'ResidFAR', 'RetailArea', 'YearBuilt', 'YearAlter1', 'ZipCode', 'YCoord', 'XCoord']]
bx1.head()

So the answer is YES, because we have the YearBuilt column that determines the year a building was built.

To answer if there are correlations between the aspects of buildings and complaints, let's make a new dataset from the 311 data where all the borough is Bronx.

In [ ]:
bx_df = df[df['Borough'] == 'BRONX']
bx_df.head()

In [ ]:
bx_df = bx_df.loc[df['Complaint Type'].isin(['HEAT/HOT WATER', 'HEATING'])]
bx_df.head()

Now that we have two modified database (bx_df & bx), let's merge it. First we drop null values on bx_df's incident adress:

In [ ]:
bx_df.dropna(subset = ["Incident Address"], axis = 0, inplace=True)
bx_df["Incident Address"].isnull().sum()

All sorted out. Now let's focus on the address with the most complaints by (again) using value_counts function:

In [ ]:
bx_df['Incident Address'].value_counts()

Let's make a cleaned, well-formatted dataframe to group the complaints by address.

In [ ]:
grp = bx_df[['Incident Address', 'Complaint Type']].groupby(['Incident Address'])
ct_bx = grp.count()
ct_bx.columns = ['Complaint Amount']
ct_bx.head()

From now further on we'll use ct_bx and bx dataframe. Let's drop duplicates from bx dataframe first.


In [ ]:
bx1.drop_duplicates(subset="Address", keep=False, inplace=True)
bx1.head()

In [ ]:
bx1['Address'].value_counts().head()

LET'S MERGE THE DATA!!!

In [ ]:
fin_df = pd.merge(ct_bx, bx1, left_index=True, right_on="Address", how="right")
fin_df.head()

In [ ]:
fin_df.info()

In [ ]:
fin_df["HOT/HEAT WATER"] = fin_df["Complaint Amount"]
fin_df.head()

Now we replace null values with 0.0 in the HEAT/HOT WATER column using nan numpy functions

In [ ]:
import numpy as np
fin_df['HOT/HEAT WATER'].replace(np.nan, 0.0, inplace = True)
fin_df.head()

We change the 0.0 values in HOT/HEAT WATER column with T/F statements to indicate whether there's a complaint or not in that particular address. Then we change the NaN values in Complaint Amount with 0.0.

In [ ]:
fin_df['HOT/HEAT WATER'] = np.where(fin_df['HOT/HEAT WATER'] >  0, True, False)
fin_df['Complaint Amount'].replace(np.nan, 0.0, inplace = True)
fin_df.head()

Now we identify columns where the YearBuilt data is 0:

In [ ]:
fin_df.loc[fin_df['YearBuilt'] == 0]

Replacing NaN values in coordinates with 00.0000

In [ ]:
import datetime as dt
today=dt.datetime.now().date()
fin_df['YCoord'] = fin_df.YCoord.fillna('00.000').astype(float)
fin_df['XCoord'] = fin_df.XCoord.fillna('00.000').astype(float) 
fin_df.head()

One last check of null values in the dataframe...

In [ ]:
fin_df.isnull().sum()

In [ ]:
fin_df['ZipCode'] = fin_df.ZipCode.fillna('00.000').astype(float)

In [ ]:
fin_df.isnull().sum()

Now we are cleared from null values. Let's make a new dataframe regarding with the building's age.

In [ ]:
df_age = fin_df[["Complaint Amount", "YearBuilt"]]
df_age = df_age[df_age.YearBuilt != 0]
df_age["Age"] = 2020 - df_age["YearBuilt"]
df_age.head()

Let's use Pearson Correlation to determine whether age have correlations with the amount of complaint.

In [ ]:
import scipy.stats as stats
stats.pearsonr(df_age["Complaint Amount"],df_age["Age"])

We can see that there's a small correlation (0.039) between the amount of complaints and building's age. Since the P value is less than 0.001 we can be confident about it. But how about all the correlations? Let's map it so we can learn about it better:

In [ ]:
import seaborn as sns
plt.figure(figsize=(20,12))
cor = fin_df.corr(method='pearson')
sns.heatmap(cor, annot = True, cmap=plt.cm.Reds)
plt.show()

Hmm, it's still a bit too much and as we can see the correlation between complaint amounts and other aspects of the field are still small. Let's simplify the study by making a new dataframe with selected columns.

In [ ]:
new_df = fin_df[["Complaint Amount","YearBuilt","BldgArea","BldgDepth","NumBldgs","NumFloors","OfficeArea"]]
#dropping 0 year built data
new_df = new_df[new_df.YearBuilt != 0]
new_df.head()

In [ ]:
#adding age column to the new_df
new_df["Age"] = 2020 - new_df["YearBuilt"]
new_df.head()

In [ ]:
#checking null values
new_df.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
cor1 = new_df.corr(method='pearson')
sns.heatmap(cor1, annot = True, cmap=plt.cm.Reds)
plt.show()

So, we can conclude that the aspect of building with the strongest correlation with the number of complaints is NumFloors (Number of Floors). We'll use this model to analyze the case deeper.

## Section 5 - Can a predictive model be built for future prediction of the possibility of complaints identified?

We'll predict the number of future complaints by building a predictive model based on the dataframe we have right now. Let's use the linear regression concept to predict the complaints over time. First, we select the appropriate data in a new, simplify, set of dataframe.

In [ ]:
dfp = df[['Borough', 'Closed Date', 'Complaint Type', 'Created Date', 'Incident Address', 'Incident Zip', 'Latitude', 'Longitude', 'Street Name', 'Status']]
dfp.head()

Select only HEAT/HOT WATER & HEATING in the BRONX borough:

In [ ]:
dfp = dfp.loc[dfp['Complaint Type'].isin(['HEAT/HOT WATER', 'HEATING'])]
dfp.head()

In [ ]:
dfp = dfp.loc[dfp['Borough'].isin(['BRONX'])]
dfp.head()

To see the yearly changes in our over 80000 complaint, let's set the created date into datetime first:

In [ ]:
dfp['Created Date'] = pd.to_datetime(dfp['Created Date'])

In [ ]:
dfp['Created Date'].head()

Use the datetime library to group the dataframe into yearly case:

In [ ]:
import datetime as dt
dfg = dfp.groupby(dfp['Created Date'].dt.year)
dfg = dfg.count()
dfg

Let's plot the yearly complaint:

In [ ]:
dfg['Complaint Type'].plot(kind = 'line')

plt.xlabel('Year')
plt.ylabel('Number of Complaints')
plt.title('Yearly Complaints of Heating Issue in the Bronx')

plt.show()

Let's eliminate the data from 2010 and 2020 since it's incomplete and might be irrelevant for our prediction. Then we define the X and y for our prediction.

In [ ]:
#defining X
X = dfg.index[1:-1]
X = [X.tolist()]
X = np.asarray(X)
X = X.reshape(-1, 1)
X

In [ ]:
#defining y
y = dfg['Complaint Type'][1:-1]
y

Into the prediction...

In [ ]:
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures 

lr = LinearRegression()

pol = PolynomialFeatures(degree = 4) 
X_pol = pol.fit_transform(X) 
 
lr.fit(X_pol, y)

Let's visualize the polynomial regression to see if it's close enough with the result...

In [ ]:
plt.scatter(X, y, color = 'green') 
plt.plot(X, lr.predict(X_pol), color = 'blue') 
 
plt.xlabel('Years') 
plt.ylabel('Number of Complaints')
plt.title('Predicted vs Actual number of complaints')
  
plt.show()

Close enough. Now let's predict the number of complaints...

In [ ]:
lr.predict(pol.fit_transform([[2020]]))

## Conclusion

1. The top complaint that 311 receives throughout the decade is Heating/Hot Water related complaint with approximately 2.15 million complaints.
2. Approximately 28.2% of the complaint comes from the Bronx borough.
3. The strongest relationship between the number of complaints with the bulding condition is with the number of floors (0.23 by Pearson correlation).
4. Using a simple polynomial regression, we can predict that the number of complaints at the end of 2020 will reach approximately 79,000 complaints.